In [ ]:
import flwr as fl
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from model import create_model

def load_data(path):
    df = pd.read_csv(path)
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    return train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
! pip install seaborn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import classification_report, recall_score, accuracy_score
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

In [ ]:
class ECGClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_test, y_test):
        self.model = model
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=1, batch_size=32, verbose=0)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.x_test, self.y_test, verbose=0)
        return loss, len(self.x_test), {"accuracy": acc}

In [ ]:
# Run client
def run_client(client_id):
    data_path = f"client{client_id}_data.csv"
    x_train, x_test, y_train, y_test = load_data(data_path)

    train_df.iloc[:, -1].value_counts().plot(kind='pie')
    plt.title('Classes Frequency')


    model = create_model()
    client = ECGClient(model, x_train, y_train, x_test, y_test)
    
    fl.client.start_numpy_client(server_address="localhost:8081", client=client)

In [ ]:
run_client(2)